In [27]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
import plotly.express as px

In [1]:
INPUT_DIR = '../input/sf-dst-predict-car-price/'
train_data = pd.read_csv(INPUT_DIR + 'train_data.csv')
#test_data = pd.read_csv(INPUT_DIR + 'test_data.csv')

display(train_data.head())
display(train_data.info())
display(train_data.describe(include='all'))

NameError: name 'pd' is not defined

### Step 0.5. Some user feature preparation

Be creative :)

In [71]:
#display(train_data['car_vin'].value_counts())

fig1_data = train_data[train_data['current_mileage'] < 1000000][['current_mileage']]
fig1 = px.box(
    fig1_data,
    x='current_mileage',
    #color='Exited',
    height=400, width=1000,
    title='Распределение пробега',
    #labels={'Balance': 'Баланс, тыс. долларов', 'Exited': 'Статус'}
    )
fig1.show()

fig2 = px.histogram(
    train_data,
    x='current_mileage',
    histfunc='count',
    nbins=40,
    #range_x=[0,90],
    height=500, width=500,
    title='Распределение пробега',
    #labels={'Age': 'Возраст, лет'}
    )
fig2.show()

Очистка данных

In [72]:
#Удаление выбросов


def cut_outliers_manual(df, column, lower_bound, upper_bound):
    mask = (df[column] > lower_bound) & (df[column] < upper_bound)
    mask = mask.apply(lambda x: not x)
    df.drop(df[mask].index, inplace=True)


def cut_outliers_iqr(df, columns):
    for column in columns:
        quartile_1, quartile_3 = df[column].quantile(0.25), df[column].quantile(0.75)    
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr * 1.5)
        upper_bound = quartile_3 + (iqr * 1.5)
        mask = (df[column] > lower_bound) & (df[column] < upper_bound)
        mask = mask.apply(lambda x: not x)
        df.drop(df[mask].index, inplace=True)

Проектирование признаков

In [73]:
#Манипуляции с VIN


def decode_vin(vin):
    #print(vin)
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        #print(vin)
        if len(vin) == 17:
            wmi = vin[0:3]
            wds = vin[3:9]
            model_year = vin[9]
            plant_code = vin[10]
        else:
            wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN    
    else:
        wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN
    
    #print(f'WMI: {wmi}, WDS: {wds}, ModelYear: {model_year}, PlantCode: {plant_code}')
    return wmi, wds, model_year, plant_code
    #return wmi

def get_vin_wmi(vin):
    #print(vin)
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        #print(vin)
        if len(vin) == 17:
            wmi = vin[0:3]
            wds = vin[3:9]
            model_year = vin[9]
            plant_code = vin[10]
        else:
            wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN    
    else:
        wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN
    
    #print(f'WMI: {wmi}, WDS: {wds}, ModelYear: {model_year}, PlantCode: {plant_code}')
    return wmi

def get_vin_wds(vin):
    #print(vin)
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        #print(vin)
        if len(vin) == 17:
            wmi = vin[0:3]
            wds = vin[3:9]
            model_year = vin[9]
            plant_code = vin[10]
        else:
            wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN    
    else:
        wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN
    
    #print(f'WMI: {wmi}, WDS: {wds}, ModelYear: {model_year}, PlantCode: {plant_code}')
    return wds

def get_vin_my(vin):
    #print(vin)
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        #print(vin)
        if len(vin) == 17:
            wmi = vin[0:3]
            wds = vin[3:9]
            model_year = vin[9]
            plant_code = vin[10]
        else:
            wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN    
    else:
        wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN
    
    #print(f'WMI: {wmi}, WDS: {wds}, ModelYear: {model_year}, PlantCode: {plant_code}')
    return model_year

def get_vin_pc(vin):
    #print(vin)
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        #print(vin)
        if len(vin) == 17:
            wmi = vin[0:3]
            wds = vin[3:9]
            model_year = vin[9]
            plant_code = vin[10]
        else:
            wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN    
    else:
        wmi, wds, model_year, plant_code = np.NaN, np.NaN, np.NaN, np.NaN
    
    #print(f'WMI: {wmi}, WDS: {wds}, ModelYear: {model_year}, PlantCode: {plant_code}')
    return plant_code

#Добавление признака 'vin_present'
def vin_present(vin):
    if vin is not np.nan:
        vin = vin.replace(' ', '')
        if len(vin) == 17:
            vin_present = True
        else:
            vin_present = False
    else:
        vin_present = False
    return vin_present

In [74]:
#Манипуляции с годом выпуска


#Добавление 'car_age'
def vehicle_age(vehicle_year, current_year=2021):
    if vehicle_year is not np.NaN:
        vehicle_age = current_year - vehicle_year
    else:
        vehicle_age = np.NaN
    
    return vehicle_age

Применение преобразований

In [75]:
#Преобразование тестового датасета
def create_expert_feats(data): 
    #Удаление выбросов
    cut_outliers_manual(data, 'current_mileage', 0, 1000000)
    cut_outliers_iqr(data, ['current_mileage'])
    
    #Манипуляции с VIN
    #data['vin_wmi'], data['vin_wds'], data['vin_my'], data['vin_pc'] = data['car_vin'].apply(decode_vin)
    data['vin_wmi'] = data['car_vin'].apply(get_vin_wmi)
    data['vin_wds'] = data['car_vin'].apply(get_vin_wds)
    data['vin_my'] = data['car_vin'].apply(get_vin_my)
    data['vin_pc'] = data['car_vin'].apply(get_vin_pc)
    #data['vin_present'] = data['car_vin'].apply(vin_present)
    
    #Манипуляции с годом выпуска
    data['vehicle_age'] = data['vehicle_year'].apply(vehicle_age)

create_expert_feats(train_data)
#create_expert_feats(test_data)

In [77]:
display(train_data.info())

display(train_data.describe(include='all'))

display(train_data.head())

#print(train_data.loc[2, 'car_vin'])

display(train_data['car_vin'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32389 entries, 0 to 34999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   row_ID                  32389 non-null  int64  
 1   vehicle_manufacturer    32389 non-null  object 
 2   vehicle_model           32384 non-null  object 
 3   vehicle_category        32389 non-null  object 
 4   current_mileage         32389 non-null  int64  
 5   vehicle_year            32389 non-null  int64  
 6   vehicle_gearbox_type    32389 non-null  object 
 7   doors_cnt               31712 non-null  object 
 8   wheels                  32389 non-null  object 
 9   vehicle_color           32021 non-null  object 
 10  vehicle_interior_color  26163 non-null  object 
 11  car_vin                 10555 non-null  object 
 12  car_leather_interior    32389 non-null  int64  
 13  deal_type               32389 non-null  object 
 14  final_price             32389 non-null

None

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,vin_wmi,vin_wds,vin_my,vin_pc,vehicle_age
count,32389.000000,32389,32384,32389,32389.000000,32389.000000,32389,31712,32389,32021,26163,10555,32389.000000,32389,3.238900e+04,10425,10425,10425,10425,32389.000000
unique,NaN,68,2147,11,NaN,NaN,4,3,2,16,11,4068,NaN,2,NaN,156,2489,30,34,NaN
top,NaN,TOYOTA,Prius,Sedan,NaN,NaN,Automatic,4/5,Left wheel,Black,Black,3VW1K7AJ6EM438747,NaN,For Sale,NaN,JTD,BD1FK7,D,A,NaN
freq,NaN,6159,2040,14250,NaN,NaN,22560,30083,29794,8530,19523,42,NaN,31834,NaN,939,157,1569,1465,NaN
mean,17491.523418,NaN,NaN,NaN,136298.586619,2010.932817,NaN,NaN,NaN,NaN,NaN,NaN,0.720461,NaN,6.068407e+03,NaN,NaN,NaN,NaN,10.067183
std,10106.643105,NaN,NaN,NaN,78758.799497,5.636080,NaN,NaN,NaN,NaN,NaN,NaN,0.448780,NaN,6.671691e+04,NaN,NaN,NaN,NaN,5.636080
min,0.000000,NaN,NaN,NaN,1.000000,1939.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.000000
25%,8714.000000,NaN,NaN,NaN,77000.000000,2009.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.500000e+03,NaN,NaN,NaN,NaN,7.000000
50%,17507.000000,NaN,NaN,NaN,129815.000000,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,4.100000e+03,NaN,NaN,NaN,NaN,9.000000
75%,26223.000000,NaN,NaN,NaN,186000.000000,2014.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,7.200000e+03,NaN,NaN,NaN,NaN,12.000000


,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price,vin_wmi,vin_wds,vin_my,vin_pc,vehicle_age
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0,NaN,NaN,NaN,NaN,7
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0,NaN,NaN,NaN,NaN,11
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0,KMH,TC6AE3,G,U,5
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0,NaN,NaN,NaN,NaN,6
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0,JTN,KHMBX7,K,1,2


 3VW1K7AJ6EM438747     42
 4T1BD1FK7EU114553     42
 4T1BD1FK6EU137614     41
 4T1BD1FK2CU027785     40
 4T1B21HK5KU013994     40
                       ..
 JTDKN3DU4A0047915      1
 5XXGN4A75EG263349      1
 2T3BFREV1JW759334      1
 4T1BF1FK1CU597366      1
 3FA6P0K98DR308742      1
Name: car_vin, Length: 4068, dtype: int64

In [78]:
fig2 = px.histogram(
    train_data,
    x='current_mileage',
    histfunc='count',
    nbins=40,
    #range_x=[0,90],
    height=500, width=500,
    title='Распределение пробега',
    #labels={'Age': 'Возраст, лет'}
    )
fig2.show()